<a href="https://www.kaggle.com/code/mikedelong/eda-with-map?scriptVersionId=137040945" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
df = pd.read_csv(filepath_or_buffer='/kaggle/input/los-angeles-crime-dataset-2020-present/Crime_Data_from_2020_to_Present.csv', 
                 parse_dates=['Date Rptd', 'DATE OCC']).drop(columns=['DR_NO'])
df.sample(n=7)

,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
368872,2021-07-10,2021-07-10,615,15,N Hollywood,1585,1,442,SHOPLIFTING - PETTY THEFT ($950 & UNDER),0325,...,IC,Invest Cont,442.0,NaN,NaN,NaN,11000 VENTURA BL,NaN,34.1430,-118.3727
683461,2023-05-08,2023-05-08,725,2,Rampart,285,1,310,BURGLARY,1822 1607 0321 0344,...,IC,Invest Cont,310.0,NaN,NaN,NaN,1100 S BURLINGTON AV,NaN,34.0488,-118.2775
296992,2021-02-18,2021-02-18,740,2,Rampart,241,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),1822 0104 0344 0325,...,IC,Invest Cont,343.0,NaN,NaN,NaN,500 S OCCIDENTAL BL,NaN,34.0668,-118.2828
489024,2022-03-13,2022-03-13,10,1,Central,159,1,510,VEHICLE - STOLEN,NaN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,1000 E 4TH PL,NaN,34.0420,-118.2340
261964,2021-09-29,2021-09-29,1000,5,Harbor,541,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329 1300,...,IC,Invest Cont,740.0,NaN,NaN,NaN,900 W CHANNEL ST,NaN,33.7538,-118.2989
320686,2021-04-21,2021-04-18,1700,19,Mission,1915,1,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),NaN,...,IC,Invest Cont,420.0,NaN,NaN,NaN,SAYRE ST,GLENOAKS BL,34.3022,-118.4427
562871,2022-12-30,2022-12-26,657,5,Harbor,558,1,330,BURGLARY FROM VEHICLE,0321 0329 0344 0358 1307 1609,...,IC,Invest Cont,330.0,NaN,NaN,NaN,00 BERTH,NaN,33.7489,-118.2798


In [2]:
df.columns

Index(['Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No',
       'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex',
       'Vict Descent', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd',
       'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2',
       'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT', 'LON'],
      dtype='object')

In [3]:
from folium import Map
from folium import Marker

sample_size = 1000
sample_df = df.sample(sample_size)

# for some reason we have to fudge the center a little
lon_fudge = -0.5
location_map = Map(location=[sample_df['LAT'].mean(), sample_df['LON'].mean() + lon_fudge], zoom_start=10)
for index, row in sample_df.iterrows():
    popup = '{}/{}/{}:{}'.format(row['LOCATION'], row['AREA NAME'], row['Crm Cd Desc'], row['Status'])
    Marker([row['LAT'], row['LON']], popup=popup).add_to(parent=location_map)

location_map

In [4]:
df.shape

(752911, 27)

In [5]:
df['Crm Cd'].nunique()

138

We have 138 codes; clearly we can't make sense of a pie with that many slices, so let's look at the top ten or so

In [6]:
crime_code_value_counts = df['Crm Cd Desc'].value_counts()
code_df = pd.DataFrame(data={'Code': crime_code_value_counts.index, 'Count': crime_code_value_counts.values}).reset_index(drop=True)
code_df.head(10)

,Code,Count
0,VEHICLE - STOLEN,80439
1,BATTERY - SIMPLE ASSAULT,59630
2,THEFT OF IDENTITY,48645
3,BURGLARY FROM VEHICLE,46526
4,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",46157
5,BURGLARY,45717
6,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",43149
7,THEFT PLAIN - PETTY ($950 & UNDER),38510
8,INTIMATE PARTNER - SIMPLE ASSAULT,37947
9,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),29310


In [7]:
import plotly.express as px
code_df.loc[code_df['Count'] < 15000, 'Code'] = 'Other' # we have too many codes so let's apply a threshold
px.pie(code_df, names='Code', values='Count').show()

In [8]:
# Surely the areas are shaped so the crime is roughly the same in each one; right?
area_value_counts = df['AREA NAME'].value_counts()
area_df = pd.DataFrame(data={'Area': area_value_counts.index, 'Count': area_value_counts.values}).reset_index(drop=True)
px.pie(area_df, names='Area', values='Count').show()